In [1]:
import json
import os
import numpy as np

from src.launcher.TV import constants
import pandas as pd
import itertools
import re
from pathlib import Path
import warnings
import seaborn as sns

In [2]:

plots_folder = constants.plots_dir['sbm_sim']

groups = ['eps', 'num_classes', 'percentage_labeled']

In [3]:
sim_id = 7
results_file_name = os.path.join(constants.results_dir['sbm_sim'][sim_id], 'comb.json')

print(os.getcwd())
if os.getcwd().endswith('TV'):
    os.chdir('../../../')
elif os.getcwd().endswith('launcher'):
    os.chdir('../../')
elif os.getcwd().endswith('src'):
    os.chdir('../')
print(os.getcwd())
with open(results_file_name) as results_file:
    results = json.load(results_file)
del results['graph_config']

results_df = pd.DataFrame(results)
results_df.columns = results_df.columns.str.replace("[+]", "p")
results_df.columns = results_df.columns.str.replace("[-]", "m")
print(results_df)

/Users/tdittrich/PycharmProjects/SignedGSPToolbox/src/launcher/TV
/Users/tdittrich/PycharmProjects/SignedGSPToolbox
       pid                                         graph_name  \
0        0  SBM with 900 nodes, 3 classes, and eps=0.15 ma...   
1        1  SBM with 900 nodes, 5 classes, and eps=0.15 ma...   
2       10  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
3      100  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
4     1000  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
...    ...                                                ...   
5932   995  SBM with 900 nodes, 10 classes, and eps=0.1999...   
5933   996  SBM with 900 nodes, 3 classes, and eps=0.25 ma...   
5934   997  SBM with 900 nodes, 5 classes, and eps=0.25 ma...   
5935   998  SBM with 900 nodes, 10 classes, and eps=0.25 m...   
5936   999  SBM with 900 nodes, 3 classes, and eps=0.3 mak...   

      percentage_labeled  num_nodes   cut_gt   eps  scale_pi  scale_pe  \
0                    0.0     

/var/folders/56/_mqv75c538730kbtd_r5qclr0000gp/T/ipykernel_93528/9870856.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  results_df.columns = results_df.columns.str.replace("[+]", "p")
/var/folders/56/_mqv75c538730kbtd_r5qclr0000gp/T/ipykernel_93528/9870856.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  results_df.columns = results_df.columns.str.replace("[-]", "m")


In [4]:
method_names = [col[len('n_err_unlabeled') + 1:] for col in results_df.columns if col.startswith('n_err_unlabeled')]
pid_list = results_df['pid']
results_df.drop(list(results_df.filter(regex = 'acc_')), axis = 1, inplace = True)
results_df.drop(list(results_df.filter(regex = 'ari_')), axis = 1, inplace = True)
results_df.drop(list(results_df.filter(regex = 'f1_')), axis = 1, inplace = True)
print(results_df)


       pid                                         graph_name  \
0        0  SBM with 900 nodes, 3 classes, and eps=0.15 ma...   
1        1  SBM with 900 nodes, 5 classes, and eps=0.15 ma...   
2       10  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
3      100  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
4     1000  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
...    ...                                                ...   
5932   995  SBM with 900 nodes, 10 classes, and eps=0.1999...   
5933   996  SBM with 900 nodes, 3 classes, and eps=0.25 ma...   
5934   997  SBM with 900 nodes, 5 classes, and eps=0.25 ma...   
5935   998  SBM with 900 nodes, 10 classes, and eps=0.25 m...   
5936   999  SBM with 900 nodes, 3 classes, and eps=0.3 mak...   

      percentage_labeled  num_nodes   cut_gt   eps  scale_pi  scale_pe  \
0                    0.0        900   4594.0  0.15         1         1   
1                    0.0        900  10864.0  0.15         1         1 

In [5]:
print(results_df['t_run_sncSponge'])
norm_results_df = results_df.copy()
for c in norm_results_df.columns:
    if c.startswith('t_run'):
        name = c[5:]
        # norm_results_df[c] /= results_df['t_run_sncSponge']
        # norm_results_df['t_run_norm' + name] = norm_results_df[c] / results_df['t_run_sncSponge']
print(norm_results_df)

mean_results = results_df.groupby(groups).mean()#.reset_index(level=list(range(1, len(groups[sim_id]))))
mean_results.drop(['pid','num_nodes'],axis = 1, inplace=True)
n_err_mean_df = mean_results.filter(regex = 'n_err_unlabeled_tv_nc_betap[0-9].[0-9]_pre0_sncSponge',axis=1)
n_err_mean_df.columns.name = 'beta'
t_run_mean_df = mean_results.filter(regex = 't_run_tv_nc_betap[0-9].[0-9]_pre0_sncSponge',axis=1)
t_run_mean_df.columns.name = 'beta'

n_err_mean_df.columns = n_err_mean_df.columns.str.replace('n_err_unlabeled_tv_nc_betap','')
n_err_mean_df.columns = n_err_mean_df.columns.str.replace('_pre0_sncSponge','').astype(float)
n_err_mean_df_compressed = n_err_mean_df.T
n_err_mean_df_compressed.columns = ['n_err'+''.join(['_{n}{v}'.format(n=n[0],v=np.round(v,2)) for n,v in zip(n_err_mean_df_compressed.columns.names,values)]) for values in n_err_mean_df_compressed.columns.values]
# n_err_mean_df = n_err_mean_df/n_err_mean_df.iloc[-1]

t_run_mean_df.columns = t_run_mean_df.columns.str.replace('t_run_tv_nc_betap','')
t_run_mean_df.columns = t_run_mean_df.columns.str.replace('_pre0_sncSponge','').astype(float)
t_run_mean_df_compressed = t_run_mean_df.T
t_run_mean_df_compressed.columns = ['t_run'+''.join(['_{n}{v}'.format(n=n[0],v=np.round(v,2)) for n,v in zip(t_run_mean_df_compressed.columns.names,values)]) for values in t_run_mean_df_compressed.columns.values]
# t_run_mean_df = t_run_mean_df/t_run_mean_df.iloc[0]

joint_mean_df = pd.concat((n_err_mean_df_compressed,t_run_mean_df_compressed),axis=1)
# joint_mean_df.index.name = 'beta'
joint_mean_df.reset_index()

n_err_mean_transpose_df = n_err_mean_df
n_err_mean_transpose_df = n_err_mean_transpose_df.unstack(level=['percentage_labeled','num_classes']).reorder_levels(['beta','num_classes','percentage_labeled'],axis=1)
n_err_mean_transpose_df.columns = ['n_err'+''.join(['_{n}{v}'.format(n=n[0],v=np.round(v,2)) for n,v in zip(n_err_mean_transpose_df.columns.names,values)]) for values in n_err_mean_transpose_df.columns.values]
# print(n_err_mean_transpose_df)
t_run_mean_transpose_df = t_run_mean_df
t_run_mean_transpose_df = t_run_mean_transpose_df.unstack(level=['percentage_labeled','num_classes']).reorder_levels(['beta','num_classes','percentage_labeled'],axis=1)
t_run_mean_transpose_df.columns = ['t_run'+''.join(['_{n}{v}'.format(n=n[0],v=np.round(v,2)) for n,v in zip(t_run_mean_transpose_df.columns.names,values)]) for values in t_run_mean_transpose_df.columns.values]
# print(t_run_mean_transpose_df)
# n_err_mean_transpose_df.index = n_err_mean_transpose_df.index.str.replace('n_err_','')
# n_err_mean_transpose_df.columns = n_err_mean_transpose_df.columns.str.cat('_n_err')
# n_err_mean_transpose_df.reset_index('eps')

joint_mean_transpose_df = pd.concat((n_err_mean_transpose_df,t_run_mean_transpose_df),axis=1)
# joint_mean_df.index.name = 'beta'
joint_mean_transpose_df.reset_index()

joint_mean_df.to_csv(os.path.join(constants.plots_dir['sbm_sim'], 'beta_pre0.csv'))
joint_mean_transpose_df.to_csv(os.path.join(constants.plots_dir['sbm_sim'], 'beta_pre0_transpose.csv'))

0       1.227048
1       1.214082
2       0.527878
3       1.708303
4       1.156433
          ...   
5932    2.618673
5933    1.539191
5934    1.403444
5935    2.474327
5936    1.539414
Name: t_run_sncSponge, Length: 5937, dtype: float64
       pid                                         graph_name  \
0        0  SBM with 900 nodes, 3 classes, and eps=0.15 ma...   
1        1  SBM with 900 nodes, 5 classes, and eps=0.15 ma...   
2       10  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
3      100  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
4     1000  SBM with 900 nodes, 5 classes, and eps=0.3 mak...   
...    ...                                                ...   
5932   995  SBM with 900 nodes, 10 classes, and eps=0.1999...   
5933   996  SBM with 900 nodes, 3 classes, and eps=0.25 ma...   
5934   997  SBM with 900 nodes, 5 classes, and eps=0.25 ma...   
5935   998  SBM with 900 nodes, 10 classes, and eps=0.25 m...   
5936   999  SBM with 900 nodes, 3 classes, and

In [6]:
n_err_high_balancedness_df = n_err_mean_df.loc[0.35]
# print(n_err_high_balancedness_df)
t_run_high_balancedness_df = t_run_mean_df.loc[0.35]
# print(t_run_high_balancedness_df)
n_err_min = n_err_high_balancedness_df.min(axis=1)
n_err_idx = n_err_high_balancedness_df.idxmin(axis=1)
# print(n_err_min)
# print(n_err_idx)
idx_df = t_run_high_balancedness_df[n_err_idx]
# print(idx_df)
diag = np.diag(idx_df)
# print(diag)
t_run_min = pd.Series(diag,index=t_run_high_balancedness_df.index)
# print(t_run_min)

n_err_last = n_err_high_balancedness_df[5.0]
t_run_last = t_run_high_balancedness_df[5.0]
# print(n_err_last)
# print(t_run_last)
diff_n_err = ((n_err_min-n_err_last)/n_err_last)
diff_t_run = ((t_run_min-t_run_last)/t_run_last)

diff_n_err.name = '$\Delta\nerr/\nerr$'
diff_t_run.name = '$\Delta\trun/\trun$'

# print(diff_t_run)
# print(diff_n_err)

diff_df = pd.concat((diff_t_run,diff_n_err),axis=1)
diff_df.index = diff_df.index.set_names(['$K$','$M$'])
diff_df.columns.name = 'val'
diff_df = diff_df.unstack('$K$')
diff_df = diff_df.reorder_levels(['$K$','val'],axis=1)
diff_df = diff_df.sort_index(axis=1)
K_level = diff_df.columns.get_level_values('$K$')
print(K_level.unique())
formatted_K_level = ['$K={K: >2d}$'.format(K=K) for K in K_level.unique()]
print(formatted_K_level)
diff_df.columns = diff_df.columns.set_levels(formatted_K_level, level='$K$', verify_integrity=False)
print(diff_df.columns)
diff_df = diff_df.T
diff_df = diff_df.reset_index()
diff_df.columns = ['','']+['$M={M:.2f}$'.format(M=M) for M in diff_df.columns[2:]]
print(diff_df)
output_path = os.path.join(constants.plots_dir['sbm_sim'])
print(diff_df.to_latex(buf=os.path.join(output_path, 'penalty_tradeoff_table.tex'),escape=False,index=False,column_format='ll|r|r|r|r',multicolumn_format='|c',multirow=True,float_format="%.2f"))

Int64Index([3, 5, 10], dtype='int64', name='$K$')
['$K= 3$', '$K= 5$', '$K=10$']
MultiIndex([('$K= 3$', '$\Delta\trun/\trun$'),
            ('$K= 3$', '$\Delta\nerr/\nerr$'),
            ('$K= 5$', '$\Delta\trun/\trun$'),
            ('$K= 5$', '$\Delta\nerr/\nerr$'),
            ('$K=10$', '$\Delta\trun/\trun$'),
            ('$K=10$', '$\Delta\nerr/\nerr$')],
           names=['$K$', 'val'])
                                  $M=0.00$    $M=3.33$   $M=10.00$  $M=20.00$
0  $K= 3$  $\Delta\trun/\trun$  147.595980  164.478622  113.032615  54.132324
1  $K= 3$  $\Delta\nerr/\nerr$   -0.220189   -0.250968   -0.256998  -0.299680
2  $K= 5$  $\Delta\trun/\trun$  454.848013  142.482936  113.630069  99.830320
3  $K= 5$  $\Delta\nerr/\nerr$   -0.279669   -0.255583   -0.269780  -0.318681
4  $K=10$  $\Delta\trun/\trun$   44.799494   42.568526   28.926574  32.095917
5  $K=10$  $\Delta\nerr/\nerr$   -0.283688   -0.321637   -0.257075  -0.274760
None


/var/folders/56/_mqv75c538730kbtd_r5qclr0000gp/T/ipykernel_93528/2344092866.py:46: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(diff_df.to_latex(buf=os.path.join(output_path, 'penalty_tradeoff_table.tex'),escape=False,index=False,column_format='ll|r|r|r|r',multicolumn_format='|c',multirow=True,float_format="%.2f"))
